In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
if torch.cuda.is_available():
    train_on_gpu = True
    print("Gpu available")
else:
    train_on_gpu = False
    print("Gpu not available")

Gpu available


In [3]:
class NN(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 64)
        self.fc2 = torch.nn.Linear(64, 32)
        self.fc3 = torch.nn.Linear(32,16)
        self.fc4 = torch.nn.Linear(16, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.softmax(self.fc4(x))
        return x
    
input_dim = 51
output_dim  = 3
batch_size = 64
model = NN(input_dim, output_dim)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model

NN(
  (fc1): Linear(in_features=51, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=3, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [153]:
# df_train = pd.read_csv("Final/train.csv")
# df_train.head()

In [4]:
class CustomLoader(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataframe = pd.read_csv(csv_file)
        self.np_input_X = self.dataframe[['Score_1', 'X_pos_1', ' Y_pos_1', ' Score_2', 'X_pos_2', ' Y_pos_2',
                                   ' Score_3', 'X_pos_3', ' Y_pos_3', ' Score_4', 'X_pos_4', ' Y_pos_4',
                                   ' Score_5', 'X_pos_5', ' Y_pos_5', ' Score_6', 'X_pos_6', ' Y_pos_6',
                                   ' Score_7', 'X_pos_7', ' Y_pos_7', ' Score_8', 'X_pos_8', ' Y_pos_8',
                                   ' Score_9', 'X_pos_9', ' Y_pos_9', ' Score_10', 'X_pos_10', ' Y_pos_10',
                                   ' Score_11', 'X_pos_11', ' Y_pos_11', ' Score_12', 'X_pos_12',
                                   ' Y_pos_12', ' Score_13', 'X_pos_13', ' Y_pos_13', ' Score_14',
                                   'X_pos_14', ' Y_pos_14', ' Score_15', 'X_pos_15', ' Y_pos_15',
                                   ' Score_16', 'X_pos_16', ' Y_pos_16', ' Score_17', 'X_pos_17',
                                   ' Y_pos_17']].to_numpy(dtype=np.float32)
        
        self.np_input_y = self.dataframe['mapped_output'].to_numpy()
        

        
#         self.input_X = (self.np_input_X).ToTensor()
#         self.input_y = torch.from_numpy(self.np_input_y)
        
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        self.input_X = torch.from_numpy(self.np_input_X)
        self.input_y = torch.from_numpy(self.np_input_y)
        
        sample = {'input': self.input_X[idx], 'output': self.input_y[idx]}
        return sample
    
trainset = CustomLoader(csv_file='Final/train.csv')
testset = CustomLoader(csv_file='Final/test.csv')

# TO Making sure data is loaded
# for i in range(len(trainset)):
#     sample = trainset[i]
#     print(sample['input'], "\n", sample['output'])
#     break
# type(sample['input'])

In [5]:
valid_size = 0.2
num_train = len(trainset)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [6]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, sampler=train_sampler)
validloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, sampler=valid_sampler)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size)

In [157]:
# number of epochs to train the model
n_epochs = 500

valid_loss_min = np.Inf # track change in validation loss
if train_on_gpu:
    model = model.cuda()
for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for content in trainloader:
#         print("Training....")
        # move tensors to GPU if CUDA is available
        data = content['input']
        target = content['output']
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for content in validloader:
#         print("Validating")
        # move tensors to GPU if CUDA is available
        data = content['input']
        target = content['output']
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(trainloader.sampler)
    valid_loss = valid_loss/len(validloader.sampler)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'output/posenet.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.237662 	Validation Loss: 1.192720
Validation loss decreased (inf --> 1.192720).  Saving model ...
Epoch: 2 	Training Loss: 0.898757 	Validation Loss: 0.791906
Validation loss decreased (1.192720 --> 0.791906).  Saving model ...
Epoch: 3 	Training Loss: 0.794928 	Validation Loss: 0.778037
Validation loss decreased (0.791906 --> 0.778037).  Saving model ...
Epoch: 4 	Training Loss: 0.776275 	Validation Loss: 0.767338
Validation loss decreased (0.778037 --> 0.767338).  Saving model ...
Epoch: 5 	Training Loss: 0.766881 	Validation Loss: 0.760804
Validation loss decreased (0.767338 --> 0.760804).  Saving model ...
Epoch: 6 	Training Loss: 0.762228 	Validation Loss: 0.756618
Validation loss decreased (0.760804 --> 0.756618).  Saving model ...
Epoch: 7 	Training Loss: 0.756975 	Validation Loss: 0.734380
Validation loss decreased (0.756618 --> 0.734380).  Saving model ...
Epoch: 8 	Training Loss: 0.704469 	Validation Loss: 0.636046
Validation loss decreased (0.73438

Epoch: 97 	Training Loss: 0.551478 	Validation Loss: 0.553148
Epoch: 98 	Training Loss: 0.551478 	Validation Loss: 0.553429
Epoch: 99 	Training Loss: 0.551476 	Validation Loss: 0.552917
Epoch: 100 	Training Loss: 0.551477 	Validation Loss: 0.553534
Epoch: 101 	Training Loss: 0.551476 	Validation Loss: 0.553244
Epoch: 102 	Training Loss: 0.551475 	Validation Loss: 0.553025
Epoch: 103 	Training Loss: 0.551475 	Validation Loss: 0.553258
Epoch: 104 	Training Loss: 0.551474 	Validation Loss: 0.553185
Epoch: 105 	Training Loss: 0.551473 	Validation Loss: 0.553325
Epoch: 106 	Training Loss: 0.551472 	Validation Loss: 0.552919
Epoch: 107 	Training Loss: 0.551471 	Validation Loss: 0.553211
Epoch: 108 	Training Loss: 0.551471 	Validation Loss: 0.553123
Epoch: 109 	Training Loss: 0.551470 	Validation Loss: 0.553125
Epoch: 110 	Training Loss: 0.551470 	Validation Loss: 0.553237
Epoch: 111 	Training Loss: 0.551469 	Validation Loss: 0.553038
Epoch: 112 	Training Loss: 0.551469 	Validation Loss: 0.55

Epoch: 222 	Training Loss: 0.551449 	Validation Loss: 0.552681
Epoch: 223 	Training Loss: 0.551449 	Validation Loss: 0.552565
Epoch: 224 	Training Loss: 0.551449 	Validation Loss: 0.552584
Epoch: 225 	Training Loss: 0.551449 	Validation Loss: 0.552589
Epoch: 226 	Training Loss: 0.551449 	Validation Loss: 0.552539
Epoch: 227 	Training Loss: 0.551449 	Validation Loss: 0.552517
Epoch: 228 	Training Loss: 0.551449 	Validation Loss: 0.552478
Epoch: 229 	Training Loss: 0.551449 	Validation Loss: 0.552694
Epoch: 230 	Training Loss: 0.551449 	Validation Loss: 0.552435
Epoch: 231 	Training Loss: 0.551448 	Validation Loss: 0.552521
Epoch: 232 	Training Loss: 0.551448 	Validation Loss: 0.552580
Epoch: 233 	Training Loss: 0.551448 	Validation Loss: 0.552426
Epoch: 234 	Training Loss: 0.551448 	Validation Loss: 0.552672
Epoch: 235 	Training Loss: 0.551448 	Validation Loss: 0.552543
Epoch: 236 	Training Loss: 0.551448 	Validation Loss: 0.552472
Epoch: 237 	Training Loss: 0.551448 	Validation Loss: 0

Epoch: 348 	Training Loss: 0.551446 	Validation Loss: 0.552246
Epoch: 349 	Training Loss: 0.551446 	Validation Loss: 0.552220
Epoch: 350 	Training Loss: 0.551446 	Validation Loss: 0.552243
Epoch: 351 	Training Loss: 0.551446 	Validation Loss: 0.552178
Epoch: 352 	Training Loss: 0.551446 	Validation Loss: 0.552256
Epoch: 353 	Training Loss: 0.551446 	Validation Loss: 0.552185
Epoch: 354 	Training Loss: 0.551446 	Validation Loss: 0.552254
Epoch: 355 	Training Loss: 0.551446 	Validation Loss: 0.552221
Epoch: 356 	Training Loss: 0.551446 	Validation Loss: 0.552205
Epoch: 357 	Training Loss: 0.551446 	Validation Loss: 0.552273
Epoch: 358 	Training Loss: 0.551446 	Validation Loss: 0.552192
Epoch: 359 	Training Loss: 0.551446 	Validation Loss: 0.552237
Epoch: 360 	Training Loss: 0.551446 	Validation Loss: 0.552286
Epoch: 361 	Training Loss: 0.551446 	Validation Loss: 0.552227
Epoch: 362 	Training Loss: 0.551446 	Validation Loss: 0.552138
Validation loss decreased (0.552140 --> 0.552138).  Sav

Epoch: 467 	Training Loss: 0.551445 	Validation Loss: 0.551970
Epoch: 468 	Training Loss: 0.551445 	Validation Loss: 0.551928
Epoch: 469 	Training Loss: 0.551445 	Validation Loss: 0.552014
Epoch: 470 	Training Loss: 0.551445 	Validation Loss: 0.551994
Epoch: 471 	Training Loss: 0.551445 	Validation Loss: 0.551964
Epoch: 472 	Training Loss: 0.551445 	Validation Loss: 0.551926
Epoch: 473 	Training Loss: 0.551445 	Validation Loss: 0.551966
Epoch: 474 	Training Loss: 0.551445 	Validation Loss: 0.551989
Epoch: 475 	Training Loss: 0.551445 	Validation Loss: 0.551978
Epoch: 476 	Training Loss: 0.551445 	Validation Loss: 0.551941
Epoch: 477 	Training Loss: 0.551445 	Validation Loss: 0.551948
Epoch: 478 	Training Loss: 0.551445 	Validation Loss: 0.551920
Epoch: 479 	Training Loss: 0.551445 	Validation Loss: 0.551919
Epoch: 480 	Training Loss: 0.551445 	Validation Loss: 0.552022
Epoch: 481 	Training Loss: 0.551445 	Validation Loss: 0.551900
Validation loss decreased (0.551917 --> 0.551900).  Sav

In [7]:
model.load_state_dict(torch.load('output/posenet.pt'))
model.eval()

NN(
  (fc1): Linear(in_features=51, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=3, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [9]:
model.eval()
if train_on_gpu:
    model = model.cuda()
count_ = list()
for content in validloader:
    # move tensors to GPU if CUDA is available
    data = content['input']
    target = content['output']
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    _, pred_labels = torch.max(output, 1)
    for i in range(0, len(target)):
        if target[i]==pred_labels[i]:
            count_.append(1)
        else:
            count_.append(0)

print("Accuracy is:", round(count_.count(1)/len(count_)*100, 2))

Accuracy is: 100.0


In [10]:
data.shape

torch.Size([46, 51])

In [11]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX file
model = onnx.load('output/posenet.onnx')

# Import the ONNX model to Tensorflow
tf_rep = prepare(model)